In [1]:
import pandas as pd
import numpy as np
import random as rn
import string
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.translate.bleu_score import sentence_bleu

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.2.0'

In [4]:
data = pd.read_csv("/kaggle/input/indiana-university-chest-xray-png/data.csv")
data.head()

,image_name,image_caption,comparison,indication,findings,impression,findings_count,impression_count,image_count
0,"CXR1_1_IM-0001-3001.png,CXR1_1_IM-0001-4001.png",xray chest pa and lateral,none,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,33,3,2
1,"CXR10_IM-0002-1001.png,CXR10_IM-0002-2001.png",pa and lateral chest x,chest radiographs,male chest pain,the cardiomediastinal silhouette is within nor...,no acute cardiopulmonary process,38,4,2
2,"CXR100_IM-0002-1001.png,CXR100_IM-0002-2001.png",chest v frontallateral pm,none,no indication,both lungs are clear and expanded heart and me...,no active disease,10,3,2
3,"CXR1000_IM-0003-2001.png,CXR1000_IM-0003-1001....",pa and lateral chest x,pa and lateral chest radiographs,male,there is increased opacity within the right up...,increased opacity in the right upper lobe with...,52,36,3
4,"CXR1001_IM-0004-1002.png,CXR1001_IM-0004-1001.png",chest v frontallateral pm,none,dyspnea subjective fevers arthritis immigrant ...,interstitial markings are diffusely prominent ...,diffuse fibrosis no visible focal acute disease,14,7,2


In [5]:
data_projecttions = pd.read_csv("/kaggle/input/image-pro/data_projections.csv")
data_projecttions.head()

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal


In [6]:
image_path = "/kaggle/input/indiana-university-chest-xray-png/NLMCXR_png/"

# Constructing data point

__limiting the data point to 2 images per data point, if we have 5 images, its 4+1 (all image + last image) so make it as 4 data points as below__

if i have 5 images then  
1 image + 5th image  
2nd image + 5th image  
3rd image + 5th image  
4th image + 5th image   
4 data point

like wise for other data point,  

if i have 3 images then  
1st + 3rd  
2nd + 3rd  
2 data point  

if i have 4 images then  
1st + 4th  
2nd  + 4th  
3rd + 4th  
3 data point  

In [7]:
data.shape

(3851, 9)

In [8]:
data['image_count'].value_counts()

2    3208
1     446
3     181
4      15
5       1
Name: image_count, dtype: int64

__Validate output:__  
2 images = 3208  
3 images = 181*2   
4 images = 15*3  
5 images = 1*4  
Total    = 3619

__Total data point___  

we should create duplicate dataframe separately to keep it in all dataset train test validate sets  
1 images = 446  
3619+446 = 4065

In [9]:
#data = data[:500]

In [10]:
def find_Fr_la(li):
    """Function to find the lateral anf frontal images
    Returns All frontal as list and Lateral as last image"""
    img_list = []
    last_img = ""
    for i in li:
        projection = data_projecttions[data_projecttions['filename'].str.contains(re.search(r"\d.*\_IM-\d.*\.", i).group())]['projection'].values
        if "Lateral" == projection:
            last_img = i
        else:
            img_list.append(i)
    return img_list, last_img

In [11]:
#Creating structured data from raw xml files
columns = ["image_1", "image_2", "impression"]
df = pd.DataFrame(columns = columns)
columns = ["image_1", "image_2", "impression"]
df_dup = pd.DataFrame(columns = columns)
no_lateral = 0
for item in tqdm(data.iterrows()):
    l = item[1]['image_name'].split(',')
    if len(l) > 2:
        li, last_img = find_Fr_la(l)
        if last_img == "":
            no_lateral +=1
            li, last_img = li[:-1], li[-1]
        for i in li:
            image_1 = i
            image_2 = last_img
            df = df.append(pd.Series([image_1, image_2, item[1]['impression']], index = columns), ignore_index = True)
    elif len(l) == 2:
        image_1 = l[0] 
        image_2 = l[1]
        df = df.append(pd.Series([image_1, image_2, item[1]['impression']], index = columns), ignore_index = True)
    elif len(l) == 1:
        #creating duplicate dataframe separately to keep it in all dataset train test validate
        df_dup = df_dup.append(pd.Series([l[0], l[0], item[1]['impression']], index = columns), ignore_index = True)
print("Total Report without Lateral images {}".format(no_lateral))

3851it [00:17, 214.76it/s]

Total Report without Lateral images 1


In [12]:
df.shape

(3532, 3)

In [13]:
df_dup.shape

(446, 3)

In [14]:
def add_start_end_token(data):
    # Combining all the above stundents 
    preprocessed_reviews_eng = []

    # tqdm is for printing the status bar
    for sentance in tqdm(data.values):
        sentance = '<start> ' + sentance + ' <end>'
        preprocessed_reviews_eng.append(sentance.strip())
    return preprocessed_reviews_eng

In [15]:
df['impression'] = add_start_end_token(df['impression'])
df_dup['impression'] = add_start_end_token(df_dup['impression'])

100%|██████████| 446/446 [00:00<00:00, 348548.46it/s]


In [16]:
df[['image_1','image_2', 'impression']].head()

,image_1,image_2,impression
0,CXR1_1_IM-0001-3001.png,CXR1_1_IM-0001-4001.png,<start> normal chest x <end>
1,CXR10_IM-0002-1001.png,CXR10_IM-0002-2001.png,<start> no acute cardiopulmonary process <end>
2,CXR100_IM-0002-1001.png,CXR100_IM-0002-2001.png,<start> no active disease <end>
3,CXR1000_IM-0003-1001.png,CXR1000_IM-0003-2001.png,<start> increased opacity in the right upper l...
4,CXR1000_IM-0003-3001.png,CXR1000_IM-0003-2001.png,<start> increased opacity in the right upper l...


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3532 entries, 0 to 3531
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_1     3532 non-null   object
 1   image_2     3532 non-null   object
 2   impression  3532 non-null   object
dtypes: object(3)
memory usage: 82.9+ KB


In [18]:
df_dup[['image_1','image_2', 'impression']].head()

,image_1,image_2,impression
0,CXR1003_IM-0005-2002.png,CXR1003_IM-0005-2002.png,<start> retrocardiac soft tissue density the a...
1,CXR1012_IM-0013-1001.png,CXR1012_IM-0013-1001.png,<start> bibasilar airspace disease and bilater...
2,CXR1024_IM-0019-1001.png,CXR1024_IM-0019-1001.png,<start> no acute abnormality <end>
3,CXR1026_IM-0021-2002.png,CXR1026_IM-0021-2002.png,<start> no acute cardiopulmonary disease <end>
4,CXR1029_IM-0022-1001.png,CXR1029_IM-0022-1001.png,<start> no pneumonia heart size normal scolios...


In [19]:
df_dup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_1     446 non-null    object
 1   image_2     446 non-null    object
 2   impression  446 non-null    object
dtypes: object(3)
memory usage: 10.6+ KB


In [20]:
image_name = []
for img in tqdm(data['image_name'].str.split(',')):
    for i in range(len(img)):
        image_name.append(img[i])

100%|██████████| 3851/3851 [00:00<00:00, 321783.90it/s]


In [21]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model

In [22]:
image_features_model = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=(299,299,3))
image_features_model.load_weights("../input/trained/trained_weights-07-0.9102.hdf5")

87916544/87910968 [==============================] - 1s 0us/step


In [23]:
img_tensor = []
for img in tqdm(image_name):
    img = tf.io.read_file(image_path + str(img))
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = preprocess_input(img)
    img_features = image_features_model(tf.constant(img)[None, :])
    img_tensor.append(img_features)

100%|██████████| 7470/7470 [15:52<00:00,  7.85it/s]


# Train Test and Validation split

In [24]:
##fixing numpy RS
np.random.seed(42)
##fixing tensorflow RS
tf.random.set_seed(32)
##python RS
rn.seed(12)

In [25]:
i_train, input_test, o_train, output_test = train_test_split(df[['image_1','image_2']].values, df['impression'].values, test_size=0.1, random_state=15)
input_train, input_val, output_train, output_val = train_test_split(i_train, o_train, test_size=0.2, random_state=15)
input_train.shape, output_train.shape, input_val.shape, output_val.shape, input_test.shape, output_test.shape

((2542, 2), (2542,), (636, 2), (636,), (354, 2), (354,))

- Train test and validation split for duplicate dataframe

In [26]:
i_train_dup, input_test_dup, o_train_dup, output_test_dup = train_test_split(df_dup[['image_1','image_2']].values, df_dup['impression'].values, test_size=0.1, random_state=15)
input_train_dup, input_val_dup, output_train_dup, output_val_dup = train_test_split(i_train_dup, o_train_dup, test_size=0.2, random_state=15)
input_train_dup.shape, output_train_dup.shape, input_val_dup.shape, output_val_dup.shape, input_test_dup.shape, output_test_dup.shape

((320, 2), (320,), (81, 2), (81,), (45, 2), (45,))

- Append duplicate data equally with train, test, and validation dataset

In [27]:
in_train = np.append(input_train, input_train_dup, axis=0)
out_train = np.append(output_train, output_train_dup, axis=0)
in_val = np.append(input_val, input_val_dup, axis=0)
out_val = np.append(output_val, output_val_dup, axis=0)
in_test = np.append(input_test, input_test_dup, axis=0)
out_test = np.append(output_test, output_test_dup, axis=0)
print("===== Final data point shape =====")
in_train.shape, out_train.shape, in_val.shape, out_val.shape, in_test.shape, out_test.shape

===== Final data point shape =====


((2862, 2), (2862,), (717, 2), (717,), (399, 2), (399,))

#### Shuffle the data point

In [28]:
# Shuffle captions and image_names together
# Set a random state
for i in range(3):
    in_train, out_train = shuffle(in_train, out_train, random_state=15)
    in_val, out_val = shuffle(in_val, out_val, random_state=15)
    in_test, out_test = shuffle(in_test, out_test, random_state=15)

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len_output = 80


tokenizer = Tokenizer(oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(out_train)
text_train = tokenizer.texts_to_sequences(out_train)
text_test = tokenizer.texts_to_sequences(out_test)
text_val = tokenizer.texts_to_sequences(out_val)
dictionary = tokenizer.word_index

word2idx = {}
idx2word = {}
for k, v in dictionary.items(): 
    word2idx[k] = v
    idx2word[v] = k

In [30]:
vocab_size = len(word2idx)+1
vocab_size

1339

In [31]:
print("===== Top 6 Word and its Index =====")
list(dictionary.items())[:6]

===== Top 6 Word and its Index =====


[('<unk>', 1),
 ('<start>', 2),
 ('<end>', 3),
 ('no', 4),
 ('acute', 5),
 ('cardiopulmonary', 6)]

In [32]:
text_output_train = pad_sequences(text_train, maxlen=max_len_output, dtype='int32', padding='post', truncating='post')
text_output_val = pad_sequences(text_val, maxlen=max_len_output, dtype='int32', padding='post', truncating='post')

In [33]:
text_output_train.shape

(2862, 80)

In [34]:
def multi_image(img, imp):
    return tf.convert_to_tensor([img_tensor[image_name.index(img[0].decode('utf-8'))], img_tensor[image_name.index(img[1].decode('utf-8'))]]), imp

In [35]:
in_train[0], text_output_train[0]

(array(['CXR1391_IM-0250-1001.png', 'CXR1391_IM-0250-5001.png'],
       dtype=object),
 array([ 2, 67, 28, 74, 88, 22,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32))

In [36]:
dataset_train = tf.data.Dataset.from_tensor_slices((in_train, text_output_train))

# Use map to load the numpy files in parallel
dataset_train = dataset_train.map(lambda item1, item2: tf.numpy_function(
          multi_image, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset_val = tf.data.Dataset.from_tensor_slices((in_val, text_output_val))

# Use map to load the numpy files in parallel
dataset_val = dataset_val.map(lambda item1, item2: tf.numpy_function(
          multi_image, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [37]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000
embedding_dim = 256
units = 128

In [38]:
# Shuffle and batch Train
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# Shuffle and batch Validation
dataset_val = dataset_val.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset_val = dataset_val.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [39]:
class Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super(Encoder, self).__init__()
        self.dense = tf.keras.layers.Dense(embedding_dim, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=45))

    def call(self, x):
        # CNN two input Images concatenate to get single vector
        # Concatenating 2 images
        # Input x shape (batch_size, 2,None,2048)
        # x1 shape (batch_size, None, 2048)
        # x2 shape (batch_size, None, 2048)
        encoder_concat = tf.keras.layers.concatenate([x[:,0], x[:,1]])
        x = self.dense(encoder_concat)
        x = tf.nn.relu(x)
        return x

In [40]:
class Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(Decoder, self).__init__()
        self.units = units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.w_1 = tf.keras.layers.Dense(units)
        self.w_2 = tf.keras.layers.Dense(units)
        # Bidirectional LSTM
        self.bilstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM \
                                      (self.units, dropout=0.3, return_sequences=True, return_state=True, \
                                       #kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), \
                                       recurrent_activation='relu', recurrent_initializer= \
                                       tf.keras.initializers.glorot_uniform(seed=26)))
        self.bilstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM\
                                      (self.units, dropout=0.2, return_sequences=True, return_state=True, \
                                       #kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), \
                                       recurrent_activation='relu', recurrent_initializer= \
                                       tf.keras.initializers.glorot_uniform(seed=26)))
        self.dense_1 = tf.keras.layers.Dense(self.units, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=45))
        self.dense_2 = tf.keras.layers.Dense(vocab_size, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=45))
        #Additive Attention
        self.attention = tf.keras.layers.AdditiveAttention(self.units)
    def call(self, x):
        # x = [dec_input, features, hidden] [decoder_input_word_tensor, encoder_output, hidden_state_previous]
        embedded_layer = self.embedding(x[0])
        x_con = tf.concat([embedded_layer, x[1]], axis=-1)
        bi_lstm = self.bilstm_1(x_con)
        lstm, forward_h, forward_c, backward_h, backward_c = self.bilstm_2(bi_lstm)
        state = tf.keras.layers.Concatenate()([forward_h, backward_h])
        state = tf.keras.layers.Concatenate()([state,x[2]])
        state = self.w_1(state)
        lstm = self.w_2(lstm)
        additive = self.attention([lstm,state])
        #decoder_1_1/additive_attention_1/Identity_1:0, shape=(None, 1, 128)
        #Reshaping to (None, 128)
        output = tf.reshape(additive, (-1, additive.shape[2]))
        output = self.dense_1(output)
        output = self.dense_2(output)
        # output will be (None, 1339)
        return output, state

In [63]:
encoder = Encoder(embedding_dim)

In [64]:
decoder = Decoder(embedding_dim, units, vocab_size)

In [65]:
optimizer = tf.keras.optimizers.Adam()
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

acc_obj = tf.keras.metrics.SparseCategoricalAccuracy()

def loss_func(real, pred):
    loss_f = loss_obj(real, pred)
    return tf.reduce_mean(loss_f)


def acc_func(real, pred):
    acc_f = acc_obj(real, pred)
    return tf.reduce_mean(acc_f)

@tf.function
def bleu_func(real, pred):
    return sentence_bleu([real], pred, weights=(1.0, 0, 0, 0))

In [66]:
!rm -r ../working

rm: cannot remove '../working': Device or resource busy


In [67]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = '../working/logs/' + current_time + '/train'
val_log_dir = '../working/logs/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

In [68]:
tf.config.experimental_run_functions_eagerly(True)
@tf.function
def train_step(tensor, target):
    loss = 0
    accuracy = 0
    # initializing the hidden state for each batch
    hidden =  tf.zeros((target.shape[0], units))
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    actual, predicted = list(), list()
    with tf.GradientTape() as tape:
        features = encoder(tensor)
        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden = decoder([dec_input, features, hidden])
            loss += loss_func(target[:, i], predictions)
            accuracy += acc_func(target[:, i], predictions)
            pred = tf.random.categorical(predictions, 1)[0][0].numpy()
            act = target[:, i][0].numpy()

            if act !=0:
                actual.append(tokenizer.index_word[act])
            if pred !=0:
                predicted.append(tokenizer.index_word[pred])
            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)
    
    total_loss = (loss / int(target.shape[1]))
    total_acc = (accuracy / int(target.shape[1]))
    total_bleu = bleu_func(actual,predicted)
    
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss, total_acc, total_bleu

#validation function
@tf.function
def val_step(tensor, target):
    loss_val = 0
    accuracy_val = 0
    # initializing the hidden state for each batch
    hidden =  tf.zeros((target.shape[0], units))
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    actual, predicted = list(), list()
    with tf.GradientTape() as tape:
        features = encoder(tensor)
        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions_val, hidden = decoder([dec_input, features, hidden])
            loss_val += loss_func(target[:, i], predictions_val)
            accuracy_val += acc_func(target[:, i], predictions_val)
            
            pred = tf.random.categorical(predictions_val, 1)[0][0].numpy()
            act = target[:, i][0].numpy()
            if act !=0:
                actual.append(tokenizer.index_word[act])
            if pred !=0:
                predicted.append(tokenizer.index_word[pred])
                
            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)
    total_loss_val = (loss_val / int(target.shape[1]))
    total_acc_val = (accuracy_val / int(target.shape[1]))
    total_bleu_val = bleu_func(actual,predicted)
    return loss_val, total_loss_val, total_acc_val, total_bleu_val

In [69]:
EPOCHS = 50
val_loss_history = []
count = 0
for epoch in range(0, EPOCHS):
    print("====== Start Epoch " +str(epoch + 1)+ " ========")
    
    total_loss_train = 0
    total_acc_train = 0
    total_bleu_train = 0
    total_loss_val = 0
    total_acc_val = 0
    total_bleu_val = 0
    print('Train loss')
    for (batch, (jpg_tensor, target)) in enumerate(dataset_train):
        batch_loss, t_loss, t_acc, t_bleu = train_step(jpg_tensor, target)
        total_loss_train += t_loss
        total_acc_train += t_acc
        total_bleu_train += t_bleu
        if batch % 40 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} acc {:.4f} bleu {:.6f}'.format(
              epoch + 1, batch, batch_loss / int(target.shape[1]), t_acc, t_bleu))
    
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', total_loss_train/ int(len(in_train) // BATCH_SIZE), step=epoch)
        tf.summary.scalar('accuracy', total_acc_train/ int(len(in_train) // BATCH_SIZE), step=epoch)
        tf.summary.scalar('bleu', total_bleu_train/ int(len(in_train) // BATCH_SIZE), step=epoch)

    #validation
    print('Validation loss')
    for (batch, (jpg_tensor, target)) in enumerate(dataset_val):
        batch_loss_val, t_loss_val, t_acc_val, t_bleu_val = val_step(jpg_tensor, target)
        total_loss_val += t_loss_val
        total_acc_val += t_acc_val
        total_bleu_val += t_bleu_val
        
        if batch % 40 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} acc {:.4f} bleu {:.6f}'.format(
              epoch + 1, batch, batch_loss_val / int(target.shape[1]), t_acc_val, t_bleu_val))
    
    with val_summary_writer.as_default():
        tf.summary.scalar('loss', total_loss_val/int(len(in_val) // BATCH_SIZE), step=epoch)
        tf.summary.scalar('accuracy', total_acc_val/int(len(in_val) // BATCH_SIZE), step=epoch)
        tf.summary.scalar('bleu', total_bleu_val/int(len(in_val) // BATCH_SIZE), step=epoch)

    print ('Epoch {}, Loss: {}, Accuracy: {}, Bleu: {}, Test Loss: {}, Test Accuracy: {}, Test Bleu: {}'.format(epoch+1, 
                            total_loss_train/ int(len(in_train) // BATCH_SIZE), 
                            (total_acc_train/ int(len(in_train) // BATCH_SIZE))*100, total_bleu_train/ int(len(in_train) // BATCH_SIZE),                                                                    
                            total_loss_val/int(len(in_val) // BATCH_SIZE), 
                            (total_acc_val/int(len(in_val) // BATCH_SIZE))*100, total_bleu_val/ int(len(in_val) // BATCH_SIZE)))
    
    val_loss_history.append(total_loss_val/int(len(in_val) // BATCH_SIZE))
    if epoch > 6:
        if count >= 4:
            print("\n\nEarlyStopping Invoked!!! Stopping the training\n\n")
            break
        else:
            if val_loss_history[epoch-1] < val_loss_history[epoch]:
                print("\n******count++ Increased******\n")
                count +=1

====== Start Epoch 1 ========
Train loss
Epoch 1 Batch 0 Loss 7.1097 acc 0.0000 bleu 0.000000
Epoch 1 Batch 40 Loss 0.6830 acc 0.8304 bleu 0.400000
Epoch 1 Batch 80 Loss 0.7917 acc 0.8498 bleu 0.222222
Validation loss
Epoch 1 Batch 0 Loss 0.6734 acc 0.8522 bleu 0.100000
Epoch 1, Loss: 1.2359944581985474, Accuracy: 80.92428588867188, Bleu: 0.12869112913276862, Test Loss: 0.9599607586860657, Test Accuracy: 89.18602752685547, Test Bleu: 0.14990489836990525
====== Start Epoch 2 ========
Train loss
Epoch 2 Batch 0 Loss 0.7833 acc 0.8546 bleu 0.040337
Epoch 2 Batch 40 Loss 0.6242 acc 0.8582 bleu 0.222222
Epoch 2 Batch 80 Loss 0.9100 acc 0.8618 bleu 0.121138
Validation loss
Epoch 2 Batch 0 Loss 1.0406 acc 0.8619 bleu 0.142857
Epoch 2, Loss: 0.8615699410438538, Accuracy: 86.8127670288086, Bleu: 0.18031143164697244, Test Loss: 0.9142494201660156, Test Accuracy: 90.11295318603516, Test Bleu: 0.1475350355721103
====== Start Epoch 3 ========
Train loss
Epoch 3 Batch 0 Loss 0.8194 acc 0.8623 bleu 0

In [70]:
!zip -r "../working/final_logs_bleu.zip" "../working/logs"

  adding: ../working/logs/ (stored 0%)
  adding: ../working/logs/20200630-090055/ (stored 0%)
  adding: ../working/logs/20200630-090055/test/ (stored 0%)
  adding: ../working/logs/20200630-090055/test/events.out.tfevents.1593507655.79c031f94454.37.32512298.v2 (deflated 69%)
  adding: ../working/logs/20200630-090055/train/ (stored 0%)
  adding: ../working/logs/20200630-090055/train/events.out.tfevents.1593507655.79c031f94454.37.32512290.v2 (deflated 69%)


In [ ]:
jkadlajsd

In [ ]:
#%load_ext tensorboard
%reload_ext tensorboard

In [ ]:
tensorboard --logdir='logs/'

In [ ]:
def get_img_tensor(image_path, img_name, model_image):
    img = tf.io.read_file(image_path + str(img_name))
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.xception.preprocess_input(img)
    img_features = model_image(tf.constant(img)[None, :])
    return img_features

In [ ]:
def evaluate(img_name):
    hidden =  tf.zeros((1, units))
    img_tensor = tf.convert_to_tensor([get_img_tensor(image_path,img_name[0], image_features_model), 
                                      get_img_tensor(image_path,img_name[1], image_features_model)])
    img_features = tf.constant(img_tensor)[None, :]
    features_val = encoder(img_features)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []
    text = ""
    for i in range(max_len_output):
        print(dec_input.shape, features_val.shape, hidden.shape)
        predictions, hidden = decoder([dec_input, features_val, hidden])
        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        if predicted_id ==0:
            word = ""
        else:
            word = tokenizer.index_word[predicted_id]
        result.append(word)
        text += " " + word
        if word == '<end>' or word == 'end':
            return result, text

        dec_input = tf.expand_dims([predicted_id], 0)
    return result, text

In [ ]:
import matplotlib.image as mpimg
from nltk.translate.bleu_score import sentence_bleu
def test_img_cap(img_data, actual_text):
    result, text = evaluate(img_data)
    """Displays images for given input array of image names"""
    fig, axs = plt.subplots(1, len(img_data), figsize = (10,10), tight_layout=True)
    count = 0
    for img, subplot in zip(img_data, axs.flatten()):
        img_=mpimg.imread(image_path+img)
        imgplot = axs[count].imshow(img_, cmap = 'bone')
        count +=1
    plt.show()
    reference = [actual_text.split()[1:-1]]
    result = result[:-1]
    print("="*50)
    print("Actual", actual_text)
    print("Predicted:",text)
    print("="*50)
    print('Individual 1-gram: {:.4f} Cumulative 1-gram: {:.4f}'.format(sentence_bleu(reference, result, weights=(1, 0, 0, 0)), sentence_bleu(reference, result, weights=(1, 0, 0, 0))))
    print('Individual 2-gram: {:.4f} Cumulative 2-gram: {:.4f}'.format(sentence_bleu(reference, result, weights=(0, 1, 0, 0)), sentence_bleu(reference, result, weights=(0.5, 0.5, 0, 0))))
    print('Individual 3-gram: {:.4f} Cumulative 3-gram: {:.4f}'.format(sentence_bleu(reference, result, weights=(0, 0, 1, 0)), sentence_bleu(reference, result, weights=(0.33, 0.33, 0.33, 0))))
    print('Individual 4-gram: {:.4f} Cumulative 4-gram: {:.4f}'.format(sentence_bleu(reference, result, weights=(0, 0, 0, 1)), sentence_bleu(reference, result, weights=(0.25, 0.25, 0.25, 0.25))))

In [ ]:
test_img_cap(in_test[164], out_test[164])

In [ ]:
test_img_cap(in_test[64], out_test[64])

In [ ]:
test_img_cap(in_test[29], out_test[29])

In [ ]:
test_img_cap(in_test[229], out_test[229])

In [ ]:
test_img_cap(in_test[365], out_test[365])

In [ ]:
test_img_cap(in_test[363], out_test[363])

In [ ]:
test_img_cap(in_test[103], out_test[103])

In [ ]:
test_img_cap(in_test[65], out_test[65])

# Conclusion

- The model build on Bidirectional LSTM with attention seems better model than than basic model
- Loss is converged to 0.3 with accuracy of 89% train and 92% validation from the result we can see there is similarity between each predicted and actual output.
- There are some major impression identified in the predicted output if there are any major actual impression.
- I have used InceptionV3 model due to its size and tensor output when compared with other imageNet trained model.
- Model feature vector is from the pretrained Inception model. weights are save and set to the while model creation.
- This feature extracted vector size(1,2048) then input to CNN encoder layer. this encoded vector input to BiLSTM layer. Attention layer gives the weights of (1,Unit_size) for each word by teacher forcing the output vector is input to decoder layer (BiLSTM layer) the predicted value and the attention weights are calculated. Below is the architecture of our model.

# Future work
- We can also modify the decoder layer with state of the art BERT Transformer instead of attention layer. 
- We can further increase the Encoder CNN layer to deep layer for improvements.
- Deep CNN encoder with Decoder Bert transformer will give better result than this.